In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-271171
Azure region: eastus2
Subscription id: 9e65f93e-bdd8-437b-b1e8-0647cd6098f7
Resource group: aml-quickstarts-271171


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity-ml-compute-cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cluster = ComputeTarget(workspace=ws,name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_v2",max_nodes=4)
    cluster = ComputeTarget.create(ws,cluster_name,compute_config)
    cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.1, 4.0),
    '--max_iter': choice(50, 100, 150, 200, 250, 300)
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('./', script='train.py', compute_target=cluster, environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8,
                                    max_concurrent_runs=4,
                                    run_config=src)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

2025-01-04 10:51:48.511675: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-04 10:51:51.084345: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-04 10:51:51.680890: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-04 10:51:58.134211: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-04 10:52:12.731251: W tensorflow/compiler/tf2

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550
Web View: https://ml.azure.com/runs/HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-271171/workspaces/quick-starts-ws-271171&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-01-04T10:50:10.8564225Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-01-04T10:50:11.5173491Z][SCHEDULER][INFO]Scheduling job, id='HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_2' 
[2025-01-04T10:50:11.3971558Z][SCHEDULER][INFO]Scheduling job, id='HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_0' 
[2025-01-04T10:50:11.5162710Z][SCHEDULER][INFO]Scheduling job, id='HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_1' 
[2025-01-04T10:50:11.5182776Z][SCHEDULER][INFO]Scheduling job, id='HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_3' 
[2025-01-04T10:50:12.1248190Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_0' 
[2025-01-04T10:5

{'runId': 'HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550',
 'target': 'udacity-ml-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-01-04T10:50:09.364586Z',
 'endTimeUtc': '2025-01-04T11:09:47.616951Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '8be09aaa-5667-469d-acb3-13212ed7f479',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_7',
  'score': '0.9147192716236724',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType':

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_7,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [6]:
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])

print(best_run_parameter_values)

Best Run Id:  HD_ba10dfd2-d83f-4c8d-a845-c6e8ff8ab550_7
Accuracy: 0.9147192716236723
['--C', '3.566313318702827', '--max_iter', '100']


In [7]:
# Just to make sure we understand HyperDrive AutoML Concept more, we will do another
# experiment where we maximize F1 Score and try to find if there is a better model or not?
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='F1_score',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=8,
                                    max_concurrent_runs=4,
                                    run_config=src)

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013
Web View: https://ml.azure.com/runs/HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013?wsid=/subscriptions/9e65f93e-bdd8-437b-b1e8-0647cd6098f7/resourcegroups/aml-quickstarts-271171/workspaces/quick-starts-ws-271171&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-01-04T11:10:28.5239172Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-01-04T11:10:28.7300888Z][SCHEDULER][INFO]Scheduling job, id='HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_0' 
[2025-01-04T11:10:28.7842706Z][SCHEDULER][INFO]Scheduling job, id='HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_1' 
[2025-01-04T11:10:28.9214384Z][SCHEDULER][INFO]Scheduling job, id='HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_2' 
[2025-01-04T11:10:28.9201798Z][SCHEDULER][INFO]Scheduling job, id='HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_3' 
[2025-01-04T11:10:29.0283274Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_1' 
[2025-01-04T11:1

{'runId': 'HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013',
 'target': 'udacity-ml-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2025-01-04T11:10:27.092534Z',
 'endTimeUtc': '2025-01-04T11:13:29.331947Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"F1_score","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '3c8830dd-ef9e-4e4c-9f00-6573419802c0',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_6',
  'score': '0.5224766751484309',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_6'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType':

In [8]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")

best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_6,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
best_run_metrics = best_run.get_metrics()
best_run_parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('F1_score:', best_run_metrics['F1_score'])

print(best_run_parameter_values)

Best Run Id:  HD_e7a1c556-c94f-48eb-b3d1-1d830e3c2013_6
F1_score: 0.5224766751484309
['--C', '1.8935054654976515', '--max_iter', '300']


In [10]:
# So basically we have now 2 runs for AutoML Hyperdrive:
# 1. Run 1: Primary metric as Accuracy with score of 91%
# 2. Run 2: Attempted Primary Metric of F1-Score with score of 0.52

# Based on these 2 experiments, we conclude that our final AutoML Config should look for 'Accuracy' as our primary metric.

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
df = ds.to_pandas_dataframe(ds)
df.head()

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}


In [20]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    compute_target=cluster,
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=5)

In [21]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()

Submitting remote run.
No run_configuration provided, running on udacity-ml-compute-cluster with default configuration
Running on remote compute: udacity-ml-compute-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ec7b7791-50fc-4dc6-9fbe-ce7360ab2fb7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [22]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
automl_run_job, best_automl_model = automl_run.get_output()
joblib.dump(value=best_automl_model, filename='best_automl_model_AUC_weighted_metric.joblib')

['best_automl_model_AUC_weighted_metric.joblib']

In [23]:
best_automl_model

PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, is_cross_validation=True, working_dir='/mnt/batch/tasks/shared/LS_root/mounts/clusters/udacity-ml-compute/code/Users/odl_user_271171')),
                                                 ('prefittedsoftvotingclassifier',
                                                  PreFittedSoftVotingClassifier(classific...s=131, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), reg_alpha=0.3684210526315789, reg_lambda=1, subsample=1))]))], flatten_transform=False, weights=[0.2857142857142857, 0.2857142857142857, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285]))],
                                       'verbose': False},
                             y_transformer={},
                             y_transformer_name='LabelEncoder')

In [24]:
automl_run_job

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ec7b7791-50fc-4dc6-9fbe-ce7360ab2fb7_35,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
cluster.delete()